In [2]:
import pandas as pd
from IPython.display import display

# tsv 파일을 DataFrame으로 읽기
user = pd.read_csv('user.tsv', sep='\t', names=['User', 'History', 'Train', 'Test'])
user_raw = pd.read_csv('user(raw).tsv', sep='\t',  names=['User', 'History', 'Train', 'Test'])

history_news = pd.read_csv('history/news.tsv', sep='\t',  names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])

train_news = pd.read_csv('train/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
train_behaviors = pd.read_csv('train/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])

test_news = pd.read_csv('test/news.tsv', sep='\t', names=['News ID', 'Publish', 'Title', 'Click time history', 'Category'])
test_behaviors = pd.read_csv('test/behaviors.tsv', sep='\t', names=['User ID', 'Click time', 'Click history', 'click'])



## User의 history,train news title 추출

In [59]:
ID = 'U4'

history = user[user['User']==ID]['History'].iloc[0]

# History에서 News ID 추출
news_ids = []
[news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

# News ID의 맞는 Title 추출
titles = []
for news_id in news_ids:
    matching_rows = history_news[history_news['News ID'] == news_id]
    if not matching_rows.empty:
        titles.append(matching_rows.iloc[0]['Title'])
        


train = user[user['User']==ID]['Train'].iloc[0]

# Train에서 News ID 추출
train_ids = []
[train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

# News ID의 맞는 Title 추출
train_titles = []
for train_id in train_ids:
    matching_rows = train_news[train_news['News ID'] == train_id]
    if not matching_rows.empty:
        train_titles.append(matching_rows.iloc[0]['Title'])


print(news_ids)
print(len(news_ids))

print(titles) 
print(len(titles))

print(train_ids) 
print(len(train_ids))

print(train_titles) 
print(len(train_titles))

['N2', 'N148', 'N206', 'N335', 'N329', 'N408', 'N868', 'N879', 'N830', 'N683', 'N1092', 'N1057', 'N1342', 'N1444']
14
['Her koker det over for Tønseth. Så stakk han fra stadion i sinne.', 'Nødbluss sendt gjennom vindu startet branntilløp', 'Næringsministeren ber ledige «ta grep»', 'Her laver snøen ned', 'Fire kjørt til sykehus', 'Kalla slet med å holde tilbake tårene: – Jeg skal ikke dele alle mine mørke tanker', 'Politiet frigir navn på kvinne som døde etter E39-ulykke', 'Navnet på druknet 40-åring er frigitt', 'Nå er navnet frigitt etter dødsfall i Gjemnes', 'Motorsagspøk ble blodig', '- Helt greit hvis bensinprisen blir 19-20 kroner', '- Helt ufattelig at det går an å behandle en ny bil på denne måten', '- Jeg fikk svingt unna i siste liten', '- Vi ble skjelvne, begge to']
14
['N2333', 'N2237', 'N2594', 'N2924']
4
['-Jeg er 90 prosent sikker på at valpen min er tatt av ulv', 'Det får da være grenser for dårlig planlegging ved kjøp av nye togvogner', 'I morgen kommer det et voldsomt 

## Negative News 추출

In [60]:
# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
history_news_sorted

,News ID,Publish,Title,Click time history,Category
0,N800,2000-11-07 15:29:05,Liker ikke harry navn,2017-01-02 21:54:18,NaN
1,N395,2001-09-10 10:01:11,50 kirkebranner påsatt,2017-01-02 02:14:53,nyheter|innenriks
2,N117,2001-12-17 13:48:43,- Ringenes Herre er ingen barnefilm,2017-01-01 09:57:07,NaN
3,N1393,2002-04-09 15:00:31,Hans Nissens gate,2017-01-04 02:32:32,nyheter|trondheim
4,N409,2002-05-24 19:33:56,Folkeliv,2017-01-02 02:37:05,NaN
...,...,...,...,...,...
1511,N1471,2017-01-03 20:30:00,Kvinner reiser utenlands for å oppfylle babydr...,"2017-01-04 06:03:34,2017-01-04 06:37:16,2017-0...",NaN
1512,N1464,2017-01-03 20:52:04,- Du skal måke fortauet utenfor huset ditt,"2017-01-04 05:54:35,2017-01-04 05:54:36,2017-0...",nyheter|trondheim
1513,N1469,2017-01-03 20:54:15,Nå blir det flere «klineseter» i kinosalen,"2017-01-04 05:59:30,2017-01-04 06:01:12,2017-0...",NaN
1514,N1490,2017-01-03 21:39:58,Ladbare hybrider på sterk fremmarsj,"2017-01-04 06:42:38,2017-01-04 06:42:41,2017-0...",nyheter|sortrondelag


In [61]:
# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_ids = []
negative_titles = []
used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

for news_id in news_ids:
    idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 뉴스를 찾기
    while above_idx >= 0:
        if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
            break
        above_idx -= 1

    # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 뉴스를 찾기
    while below_idx < len(history_news_sorted):
        if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
            break
        below_idx += 1

    negative_ids.append(similar_ids)
    negative_titles.append(similar_titles)


print(negative_ids)  
print(negative_titles)  

[['N1', 'N588'], ['N577', 'N98'], ['N217', 'N1214'], ['N333', 'N350'], ['N322', 'N358'], ['N389', 'N426'], ['N843', 'N874'], ['N878', 'N905'], ['N831', 'N837'], ['N682', 'N693'], ['N1097', 'N1098'], ['N1053', 'N1066'], ['N1304', 'N1337'], ['N1401', 'N1412']]
[['Se lesernes nyttårsbilder', 'Kong Harald: - Litt uvirkelig å bli 80'], ['16-åring skadd i ulykke med fyrverkeri', 'Startet det nye året med fyrverkerishow'], ['Kristine (26) trente seg i senk: - Hadde en stemme i hodet', 'Får starte 2017 i sitt eget hjem'], ['Norsk seier i nyttårshopprennet: - For en prestasjon', 'Klarer du å synge Creedence-låter i to strake timer?'], ['- Istanbul vekker vonde minner om det som skjedde i Paris', 'Frykter at gode fiskeplasser går tapt til Salmars nye havmerd'], ['Iiiiiiiiiskaldt nyttårsbad', 'Prøvekjørt: Skoda Superb. Plassmesteren'], ['Nå er RBK-floppen legende i Australia', 'I kveld kan du få et spesielt syn på himmelen'], ['- Snåsamannen har spart samfunnet for mange millioner', 'Så viktige e

# Positive & Negative 

In [62]:
number = len(titles)

print('[Click History]')
for i in range(number):
    print(f'{i+1}. click : {titles[i]} / non-click_1: {negative_titles[i][0]}, non-click_2: {negative_titles[i][1]}')
    
print()
print('[Question]')

for i, title in enumerate(train_titles):
    print(f"{i+1}. Do you think the user will read the news '{title}'?")


[Click History]
1. click : Her koker det over for Tønseth. Så stakk han fra stadion i sinne. / non-click_1: Se lesernes nyttårsbilder, non-click_2: Kong Harald: - Litt uvirkelig å bli 80
2. click : Nødbluss sendt gjennom vindu startet branntilløp / non-click_1: 16-åring skadd i ulykke med fyrverkeri, non-click_2: Startet det nye året med fyrverkerishow
3. click : Næringsministeren ber ledige «ta grep» / non-click_1: Kristine (26) trente seg i senk: - Hadde en stemme i hodet, non-click_2: Får starte 2017 i sitt eget hjem
4. click : Her laver snøen ned / non-click_1: Norsk seier i nyttårshopprennet: - For en prestasjon, non-click_2: Klarer du å synge Creedence-låter i to strake timer?
5. click : Fire kjørt til sykehus / non-click_1: - Istanbul vekker vonde minner om det som skjedde i Paris, non-click_2: Frykter at gode fiskeplasser går tapt til Salmars nye havmerd
6. click : Kalla slet med å holde tilbake tårene: – Jeg skal ikke dele alle mine mørke tanker / non-click_1: Iiiiiiiiiskaldt 

# 한번에 출력

In [94]:
ID = 'U33'

history = user[user['User']==ID]['History'].iloc[0]

# History에서 News ID 추출
news_ids = []
[news_ids.append(entry.split(',')[0]) for entry in history.split(';') if entry and entry.split(',')[0] not in news_ids]

# News ID의 맞는 Title 추출
titles = []
for news_id in news_ids:
    matching_rows = history_news[history_news['News ID'] == news_id]
    if not matching_rows.empty:
        titles.append(matching_rows.iloc[0]['Title'])
        


train = user[user['User']==ID]['Train'].iloc[0]

# Train에서 News ID 추출
train_ids = []
[train_ids.append(entry.split(',')[0]) for entry in train.split(';') if entry and entry.split(',')[0] not in train_ids]

# News ID의 맞는 Title 추출
train_titles = []
for train_id in train_ids:
    matching_rows = train_news[train_news['News ID'] == train_id]
    if not matching_rows.empty:
        train_titles.append(matching_rows.iloc[0]['Title'])


# publish 순서에 맞게 오름차순으로 정렬
history_news_sorted = history_news.sort_values(by='Publish', ascending=True).reset_index(drop=True)
history_news_sorted


# news_id에서 각 news_id에 대한 유사한 publish 시간 뉴스 기사 찾기
negative_ids = []
negative_titles = []
used_ids = set(news_ids)  # 중복을 방지하기 위해 사용된 뉴스 ID를 추적

for news_id in news_ids:
    idx = history_news_sorted[history_news_sorted['News ID'] == news_id].index[0]
    above_idx, below_idx = idx - 1, idx + 1
    similar_ids = []
    similar_titles = []

    # 이미 존재하는 news_id와 중복을 제외하고 위에서 가장 가까운 뉴스를 찾기
    while above_idx >= 0:
        if history_news_sorted.loc[above_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[above_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[above_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[above_idx, 'News ID'])
            break
        above_idx -= 1

    # 이미 존재하는 news_id와 중복을 제외하고 아래에서 가장 가까운 뉴스를 찾기
    while below_idx < len(history_news_sorted):
        if history_news_sorted.loc[below_idx, 'News ID'] not in used_ids:
            similar_ids.append(history_news_sorted.loc[below_idx, 'News ID'])
            similar_titles.append(history_news_sorted.loc[below_idx, 'Title'])
            used_ids.add(history_news_sorted.loc[below_idx, 'News ID'])
            break
        below_idx += 1

    negative_ids.append(similar_ids)
    negative_titles.append(similar_titles)


number = len(titles)

print('[Click History]')
for i in range(number):
    print(f'{i+1}. click : {titles[i]} / non-click_1: {negative_titles[i][0]}, non-click_2: {negative_titles[i][1]}')
    # print(f'{i+1}. click : {titles[i]} / non-click: {negative_titles[i]}')
    
print()
print('[Question]')

for i, title in enumerate(train_titles):
    print(f"{i+1}. Do you think the user will read the news '{title}'?")



[Click History]
1. click : Stina Nilsson parkerte de norske jentene på sprinten / non-click_1: For første gang kan folk kjøpe dette på nyttårsaften, non-click_2: Russer viste monsterkrefter: - Noe av det råeste jeg har sett
2. click : Fire kjørt til sykehus / non-click_1: Norsk seier i nyttårshopprennet: - For en prestasjon, non-click_2: Her laver snøen ned
3. click : Nødbluss sendt gjennom vindu startet branntilløp / non-click_1: 16-åring skadd i ulykke med fyrverkeri, non-click_2: Startet det nye året med fyrverkerishow
4. click : I morgen blir det kaffe og boller på stasjonen / non-click_1: Emil Iversen sender bilder til Ustjugov på nettet, non-click_2: To av tre Northug-brødre tilbake for fullt: Til Lahti for å kjempe om VM-plass
5. click : 15-åringer utløste brannalarmen med fyverkeri / non-click_1: Solberg hyller dristige nordmenn, non-click_2: Mann døde etter slåsskamp i Eigersund
6. click : Kvinne omkom i ulykke på E39 / non-click_1: Hopplegenden ble rørt av Tandes triumf: – Li